<a href="https://colab.research.google.com/github/ymkge/competiton/blob/main/pytorch_fakenews_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ROOT = "/content/drive/MyDrive/python/cpt-fakenews"
CUR_DIR = ROOT+"/baseline/code"
%cd $CUR_DIR

/content/drive/MyDrive/python/cpt-fakenews/baseline/code


In [ ]:
!pip install GPUtil
!pip install transformers==4.12.2
!pip install fugashi
!pip install mecab-python3
!pip install ipadic

!pip install colorama

  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=43e056580eb1a220c22dab4796088ef31988c5c942bb8fba826c78e080395b9e
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil
     |████████████████████████████████| 3.1 MB 15.1 MB/s 
     |████████████████████████████████| 3.3 MB 82.4 MB/s 
     |████████████████████████████████| 895 kB 90.6 MB/s 
     |████████████████████████████████| 67 kB 7.0 MB/s 
     |████████████████████████████████| 596 kB 85.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 568 kB 15.7 MB/s 
     |████████████████████████████████| 574 kB 14.2 MB/s 
     |████████████████████████████████| 13.4 MB 14.4 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=fcd8c673ff4e6839a1

In [ ]:
import os
import gc
import copy
import time
import random
import string

import datetime

# For data manipulation
import numpy as np
import pandas as pd

from glob import glob

# Pytorch Imports
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

# For Transformer Models
from transformers import AutoTokenizer, AutoModel, AdamW

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import pickle
import re
import unicodedata


import GPUtil
import regex
import scipy as sp
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import StratifiedKFold
import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertJapaneseTokenizer, BertForSequenceClassification, AdamW
  
warnings.filterwarnings('ignore')

In [ ]:
exp = "epoch20_baseline"

CONFIG = {"seed": 2022,
          "model_name": r'cl-tohoku/bert-base-japanese-whole-word-masking',  # r'cl-tohoku/bert-base-japanese-v2'
          "train_batch_size": 32,
          "test_batch_size": 64,
          "max_length": 128,
          "num_classes": 2,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          }

CONFIG["tokenizer"] = AutoTokenizer.from_pretrained(CONFIG['model_name'])

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

In [ ]:
!ls $ROOT/baseline/models

bert-2022-03-03-05-38  bert-2022-03-11-12-21  bert-2022-03-18-16-52
bert-2022-03-11-00-10  bert-2022-03-18-15-22  bert-2022-03-28-18-54


In [ ]:
name = "bert-2022-03-28-18-54"
MODEL_PATHS = glob(f'{ROOT}/baseline/models/{name}/*.bin')
MODEL_PATHS.sort()

In [ ]:
MODEL_PATHS

['/content/drive/MyDrive/python/cpt-fakenews/baseline/models/bert-2022-03-28-18-54/Loss-Fold-0.bin',
 '/content/drive/MyDrive/python/cpt-fakenews/baseline/models/bert-2022-03-28-18-54/Loss-Fold-1.bin',
 '/content/drive/MyDrive/python/cpt-fakenews/baseline/models/bert-2022-03-28-18-54/Loss-Fold-2.bin',
 '/content/drive/MyDrive/python/cpt-fakenews/baseline/models/bert-2022-03-28-18-54/Loss-Fold-3.bin',
 '/content/drive/MyDrive/python/cpt-fakenews/baseline/models/bert-2022-03-28-18-54/Loss-Fold-4.bin']

In [ ]:
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [ ]:
df = pd.read_csv(ROOT + "/data/test.csv")
submission = pd.read_csv(ROOT +"/data/sample_submission.csv")

In [ ]:
class FakenewsDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df['text'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
    
        text = self.text[index]
        inputs_text = self.tokenizer.encode_plus(
                                text,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        
        ids = inputs_text['input_ids']
        mask = inputs_text['attention_mask']
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
        }

In [ ]:
test_dataset = FakenewsDataset(df, CONFIG['tokenizer'], max_length=CONFIG['max_length'])
test_loader = DataLoader(test_dataset, batch_size=CONFIG['test_batch_size'],
                         num_workers=2, shuffle=False, pin_memory=True)

In [ ]:
class FakenewsModel(nn.Module):
    def __init__(self, model_name):
        super(FakenewsModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(768, CONFIG['num_classes'])
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, ids, mask):        
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        out = self.drop(out[1])
        outputs = self.fc(out)
        outputs = self.sigmoid(outputs)
        
        return outputs.squeeze()

In [ ]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        
        outputs = model(ids, mask)
        
        PREDS.append(outputs.cpu().detach().numpy()) 
    
    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return PREDS

In [ ]:
def inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = FakenewsModel(CONFIG['model_name'])
        model.to(CONFIG['device'])
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    return final_preds

In [ ]:
preds = inference(MODEL_PATHS, test_loader, CONFIG['device'])

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 60/60 [00:15<00:00,  3.87it/s]
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 2


...

100%|██████████| 60/60 [00:15<00:00,  3.91it/s]
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 4


100%|██████████| 60/60 [00:15<00:00,  3.91it/s]
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 5


100%|██████████| 60/60 [00:15<00:00,  3.91it/s]


In [ ]:
preds

array([[0.04572206, 0.95790803],
       [0.930957  , 0.06885507],
       [0.12207655, 0.87459964],
       ...,
       [0.03529278, 0.96308357],
       [0.04786194, 0.9559719 ],
       [0.22466628, 0.7658968 ]], dtype=float32)

In [ ]:
df['preds'] = np.argmax(preds, axis=1)
df.head()

,id,text,preds
0,5944,時事通信によると、石油元売り最大手の新日本石油(新日石)は3月4日、九州石油(九石本社:東京...,1
1,1129,7月23日に告示、30日に投開票された和歌山市長選挙で、現職の大橋建一氏(60歳)が再びかじ...,0
2,6013,読売新聞と朝日新聞によると14日、関東地方の広い範囲で停電が発生し、各交通機関に影響があった...,1
3,4204,時事通信の報道によると小沢一郎・民主党代表は5日午後、党本部で緊急記者会見を行い代表職を辞め...,1
4,5792,岩手日報、デイリースポーツによると、7月17日早朝、岩手県洋野町の山林で「母娘4-5人を刺し...,1


In [ ]:
df["predisFake"] = np.argmax(preds, axis=1)

In [ ]:
df["predisFake"].value_counts()

0    1969
1    1812
Name: predisFake, dtype: int64

In [ ]:
submission["isFake"] = df["predisFake"]
submission.to_csv(ROOT+f"/baseline/submission_{exp}.csv", index=False)